Action performed:
- PDFminer for pdf to txt conversion, for all input pdfs
- From txt, description using nltk
- summarization using nltk

In [1]:
# general imports
from pathlib import Path
import os

# processing imports
import pandas as pd
import summa

In [2]:
# utils elements to move to utils after development

def getListOfFiles(dirName):
    '''
        For the given path, get the List of all files in the directory tree 
    '''
    paths = []
    for path, subdirs, files in os.walk(dirName):
        for name in files:
            paths.append((Path(path+name)))            
    return paths

from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTPage, LTChar, LTAnno, LAParams, LTTextBox, LTTextLine

class PDFPageDetailedAggregator(PDFPageAggregator):
    def __init__(self, rsrcmgr, pageno=1, laparams=None):
        PDFPageAggregator.__init__(self, rsrcmgr, pageno=pageno, laparams=laparams)
        self.rows = []
        self.page_number = 0
    def receive_layout(self, ltpage):        
        def render(item, page_number):
            if isinstance(item, LTPage) or isinstance(item, LTTextBox):
                for child in item:
                    render(child, page_number)
            elif isinstance(item, LTTextLine):
                child_str = ''
                for child in item:
                    if isinstance(child, (LTChar, LTAnno)):
                        child_str += child.get_text()
                child_str = ' '.join(child_str.split()).strip()
                if child_str:
                    row = (page_number, item.bbox[0], item.bbox[1], item.bbox[2], item.bbox[3], child_str) # bbox == (x1, y1, x2, y2)
                    self.rows.append(row)
                for child in item:
                    render(child, page_number)
            return
        render(ltpage, self.page_number)
        self.page_number += 1
        self.rows = sorted(self.rows, key = lambda x: (x[0], -x[2]))
        self.result = ltpage

from collections import OrderedDict
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams

# TODO: deal with words cut in half "pro- pagation of..."
def convert(input_file, pages_range):
    """
    :param input_file: PDF filename
    :param pages_range: (nb_first_page_rse:int, nb_last_page_rse:int), starting at 1
    """
    fp = open(input_file, 'rb')
    parser = PDFParser(fp)
    doc = PDFDocument(parser)
    # doc.initialize("passwrd") # leave empty for no password

    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageDetailedAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    pages_selection = range(pages_range[0]-1,(pages_range[1]-1)+1)
    for nb_page_parsed, page in enumerate(PDFPage.create_pages(doc)):
        if nb_page_parsed in pages_selection:
            interpreter.process_page(page)
            # receive the LTPage object for this page
            device.get_result()

    # GROUPING
    grouped_text = OrderedDict() # keep order is of identification in the document.
    for (page_nb, x_min, y_min, x_max, y_max, text) in device.rows:
        page_nb = (pages_range[0]) + page_nb # elsewise device starts again at 0
        if page_nb not in grouped_text.keys():
            grouped_text[page_nb] = {}
        x_min = round(x_min)//10 # manipulate the level of aggregation,(unit?) --> x_min might be slighly adapted
        try:
            grouped_text[page_nb][x_min]+= " " + text # TODO: check if anomaly in paragraphe here
        except:
            grouped_text[page_nb][x_min] = text
    
    return grouped_text
    
###     KEEP for FUTURE  AGGREGATION
#     # FULL TEXT BY PAGE
#     text_by_page = OrderedDict()
#     for key, values in grouped_text.items():
#         if values:
#             text_by_page[key] = ""
#             for _, text in values.items():
#                 text_by_page[key] += text + " "
#     # FULL TEXT
#     pdf_full_txt = ""
#     for key, text in text_by_page.items():
#         pdf_full_txt += text + " "
        
#     return (pdf_full_txt, text_by_page)

In [3]:
entreprises_filename = "../../data/input/Entreprises/entreprises.csv"
df_entreprises = pd.read_csv(entreprises_filename, sep=";")
pd.read_csv(entreprises_filename, sep=";").set_index("project_denomination").T.to_dict()

{'vinci': {'SIREN': 552037806,
  'denomination': 'VINCI',
  'rse_ranges': '(38,48)|(207,266)'},
 'eiffage': {'SIREN': 709802094,
  'denomination': 'Eiffage',
  'rse_ranges': '(125,202)'},
 'bouygues': {'SIREN': 397480930,
  'denomination': 'Bouygues',
  'rse_ranges': '(98,124)'},
 'saintgobain': {'SIREN': 542039532,
  'denomination': 'Saint-Gobain',
  'rse_ranges': '(76,79)|(101,104)|(329,332)'}}

In [4]:
# filepaths
entreprises_filename = "../../data/input/Entreprises/entreprises.csv"
input_path = "../../data/input/DPEFs/Construction/"
output_filename= "../../data/processed/DPEFs/dpef_paragraphs.csv"

# Entreprises : data
dict_entreprises = pd.read_csv(entreprises_filename, sep=";").set_index("project_denomination").T.to_dict()
# Looks like 'vinci': {'SIREN': 552037806, 'denomination': 'VINCI'},

# DPEF
all_input_files = getListOfFiles(input_path)
all_input_files = [p for p in all_input_files if p.name.lower().endswith(".pdf")]

# output has shape...
df_parsed_data = pd.DataFrame(columns = ["SIREN",
                                         "project_denomination",
                                         "pdf_name",
                                         "page_nb_in_pdf",
                                         "paragraph_id", # x_min for now
                                         "paragraph"])
for i, input_file in enumerate(all_input_files):
    if input_file.name.endswith("pdf"):
        project_denomination = input_file.name.split("\\")[-1].split("_")[0] # first word of pdf name
        print("Processing {}/{} {} [{}]".format(i,len(all_input_files),project_denomination, input_file.name))
        for rse_ranges in dict_entreprises[project_denomination]["rse_ranges"].split("|"):
            rse_ranges = eval(rse_ranges) # tuple format str to actual tuple
            print("Pages: {} to {}".format(rse_ranges[0], rse_ranges[1]))
            grouped_text = convert(input_file, rse_ranges)
            for page_nb, page_content in grouped_text.items():
                for paragraph_id, paragraph in page_content.items():
                    df_update = {"SIREN": dict_entreprises[project_denomination]["SIREN"],
                                 "denomination": dict_entreprises[project_denomination]["denomination"],
                                 "project_denomination": project_denomination,
                                 "pdf_name": input_file.name.split("\\")[-1],
                                 "page_nb_in_pdf":page_nb,
                                 "paragraph_id":paragraph_id,
                                 "paragraph":paragraph}
                    df_parsed_data = df_parsed_data.append(df_update, ignore_index=True)
df_parsed_data.to_csv(output_filename,sep=";", index=False)

Processing 0/4 bouygues [bouygues_2018_ddr.pdf]
Pages: 98 to 124
Processing 1/4 eiffage [eiffage_2018_ddr.pdf]
Pages: 125 to 202
Processing 2/4 saintgobain [saintgobain_2018_ddr.pdf]
Pages: 76 to 79
Pages: 101 to 104
Pages: 329 to 332
Processing 3/4 vinci [vinci_2018_ddr.pdf]
Pages: 38 to 48
Pages: 207 to 266


PermissionError: [Errno 13] Permission denied: '../../data/processed/DPEFs/dpef_paragraphs.csv'

In [ ]:
df_parsed_data.to_csv(output_filename,sep=";", index=False)

In [ ]:
output_filename= "../../data/processed/DPEFs/dpef_paragraphs.csv"
df_parsed_data.to_csv(output_filename,sep=";", index=False)

In [ ]:
df_parsed_data[df_parsed_data.project_denomination == "bouygues"][["paragraph_id","paragraph"]]

In [25]:
# from summa.summarizer import summarize
summa.summarizer.summarize("Des bâtiments performants énergétiquement, offrant une haute qualité d’usage à leurs habitants Eiffage Énergie Systèmes a présenté début mars 2018 son dispositif de supervision numérique énergétique des bâtiments publics, une première en Europe. Eiffage Énergie Systèmes réalise ainsi le système de supervision de la Ville de Paris. Les équipes de la branche sont chargées de la rénovation des installations techniques – instrumentation, compteurs, tableaux électriques, automates –, d’une partie des centres thermiques et de la mise en place d’un système de supervision pour l’ensemble du parc qui s’interfacera avec les outils de gestion interne existants. Afin d’affiner les données collectées, les équipes installent des cap- teurs intelligents sur les centres thermiques les plus importants et sont chargées du développement et de l’installation du logiciel pour organiser et mettre à disposition les données collectées. Par la collecte et l’exploitation de données via un réseau spécifique Internet des Objets (IoT), accompagnées d’une maintenance préventive et curative, ce dispositif favorisera une véritable démarche de management de l’énergie, et créera de nouveaux services et applications à destination des usagers. Par ailleurs, Eiffage Énergie Systèmes a signé en 2010 un contrat de performance énergétique (CPE) avec la région Centre, sur l’exploi- tation des installations énergétiques de 19 lycées durant quinze ans. L’entreprise s’est engagée à réaliser des économies d’énergie de l’ordre de 35 % et une réduction des gaz à effet de serre de 49 % d’ici à 2025. Leader français de la cogénération dans le maraîchage Eiffage Énergie Systèmes est le leader français de la cogénération dans le maraîchage, avec 116 installations en France pour 500 MWh électriques et 500 MWh thermiques installés. Ce procédé récupère le CO2 afin de favoriser la photosynthèse et produire de l’électricité, revendue par les maraîchers. La conception, l’installation et l’exploitation-maintenance de ces centrales de cogénération sont assurées par la branche et certifiées ISO 50001. Nouvelles mobilités et covoiturage Eiffage Énergie Systèmes est un moteur du développement des nouvelles mobilités, comme l’atteste le test dès janvier 2019 de la première navette autonome capable de circuler sur route ouverte en zone d’activités, baptisée Mia. Les équipes participent égale- ment au développement des réseaux de bornes de recharge élec- trique en milieu urbain comme sur autoroute. Présentée le 27 novembre par Eiffage Énergie Systèmes et ses par- tenaires au salon Pollutec, le Salon des solutions environnementales pour l’industrie, la ville et les territoires, la navette autonome Mia a;Eiffage", 
                           words=100, 
                           split=True, 
                           language="french")

['Leader français de la cogénération dans le maraîchage Eiffage Énergie Systèmes est le leader français de la cogénération dans le maraîchage, avec 116 installations en France pour 500 MWh électriques et 500 MWh thermiques installés.',
 'Nouvelles mobilités et covoiturage Eiffage Énergie Systèmes est un moteur du développement des nouvelles mobilités, comme l’atteste le test dès janvier 2019 de la première navette autonome capable de circuler sur route ouverte en zone d’activités, baptisée Mia. Les équipes participent égale- ment au développement des réseaux de bornes de recharge élec- trique en milieu urbain comme sur autoroute.',
 'Présentée le 27 novembre par Eiffage Énergie Systèmes et ses par- tenaires au salon Pollutec, le Salon des solutions environnementales pour l’industrie, la ville et les territoires, la navette autonome Mia a;Eiffage']

In [85]:
len(pdf_full_txt)

1547176